<a href="https://colab.research.google.com/github/JanXanevo/PrototypeRecommendations/blob/main/10_BestProductsForReco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

DATEN-IMPORT:

In [ ]:
# Data
dfGarten = pd.read_csv('/content/drive/MyDrive/BAHAG/Data/Garten_WGR_PNK.csv')
dfOnline = pd.read_csv('/content/drive/MyDrive/BAHAG/Data/onlineGardenProducts.csv')
dfOffline = pd.read_csv('/content/drive/MyDrive/BAHAG/Data/offlineGardenProducts.csv')

# Projects PNK
dfProjectsOnlinePNK = pd.read_csv('/content/drive/MyDrive/BAHAG/Data/projectsGardenOnline.csv')
dfProjectsOfflinePNK = pd.read_csv('/content/drive/MyDrive/BAHAG/Data/projectsGardenOffline.csv')

# Projects ProduktNr ?

In [ ]:
# online

top_PNKOnline = dfOnline['PRODUCT_NAME_CATEGORY'].value_counts().head(40)
top_produktNrOnline = dfOnline['produktnr'].value_counts().head(10)
top_produktNameOnline = dfOnline['SAP_NAME'].value_counts().head(500)

In [ ]:
# offline

top_PNKOffline = dfOffline['PRODUCT_NAME_CATEGORY'].value_counts().head(40)
top_produktNrOffline = dfOffline['produktnr'].value_counts().head(10)
top_produktNameOffline = dfOffline['SAP_NAME'].value_counts().head(500)

ANALYSE:

In [ ]:
# Überlappung der Ergebnisse von Online & Offline
# Vorsicht: Offline steht hierbei viel stärker im Gewicht bei der Gesamtmenge am Ende

# ProduktName
set_offlineName = set(top_produktNameOffline.index)
set_onlineName = set(top_produktNameOnline.index)

overlap_count = {value: dfOffline['SAP_NAME'].isin([value]).sum() + dfOnline['SAP_NAME'].isin([value]).sum() for value in set_offlineName.intersection(set_onlineName)}
sorted_overlapName = pd.DataFrame(sorted(overlap_count.items(), key=lambda x: x[1], reverse=True))
sorted_overlapName.columns = ['SAP_NAME', 'Anzahl']

sorted_overlapName

In [ ]:
# PNK
set_offlinePNK = set(top_PNKOffline.index)
set_onlinePNK = set(top_PNKOnline.index)

overlap_count = {value: dfOffline['PRODUCT_NAME_CATEGORY'].isin([value]).sum() + dfOnline['PRODUCT_NAME_CATEGORY'].isin([value]).sum() for value in set_offlinePNK.intersection(set_onlinePNK)}
sorted_overlapPNK = pd.DataFrame(sorted(overlap_count.items(), key=lambda x: x[1], reverse=True))
sorted_overlapPNK.columns = ['Key', 'Anzahl']

sorted_overlapPNK

In [ ]:
# Erstelle leere Listen für die Überschneidungen, die entsprechenden Keys und Projekte
overlap_list = []
matching_keys_list = []
matching_projects_list = []

# Durchlaufe die Zeilen in df2
for index, row in dfProjectsOnlinePNK.iterrows():
    # Überprüfe auf Überschneidung und speichere die übereinstimmenden Keys
    matching_keys = [key for key in sorted_overlapPNK['Key'] if key in row['VerbundeneKnoten']]
    overlap = len(matching_keys) > 0

    overlap_list.append(overlap)
    matching_keys_list.append(matching_keys if overlap else None)

    # Wenn eine Überschneidung vorliegt, speichere das Projekt
    matching_project = row['Projekt'] if overlap else None
    matching_projects_list.append(matching_project)

# Füge Spalten mit den Überschneidungen, den entsprechenden Keys und Projekten zum DataFrame 2 hinzu
dfProjectsOnlinePNK['is_top_PNK'] = overlap_list
dfProjectsOnlinePNK['ÜbereinstimmenderKey'] = matching_keys_list

print(dfProjectsOnlinePNK)

In [ ]:
mergedTest = pd.merge(sorted_overlapName, dfGarten, on='SAP_NAME', how='inner')
mergedTest

In [ ]:
# Erstelle leere Listen für die Überschneidungen, die entsprechenden Keys und Projekte
overlap_list = []
matching_keys_list = []
matching_projects_list = []

# Durchlaufe die Zeilen in df2
for index, row in dfProjectsOnlinePNK.iterrows():
    # Überprüfe auf Überschneidung und speichere die übereinstimmenden Keys
    if row['ÜbereinstimmenderKey'] is not None:
        matching_sap_name = mergedTest.loc[mergedTest['PRODUCT_NAME_CATEGORY'].isin(row['ÜbereinstimmenderKey']), 'SAP_NAME']

        if not matching_sap_name.empty:
            matching_sap_name = matching_sap_name.iloc[0]
            overlap = True
        else:
            matching_sap_name = None
            overlap = False

        matching_keys_list.append(matching_sap_name if overlap else None)
    else:
      matching_sap_name = ""
      matching_keys_list.append(None)
      overlap = len(matching_sap_name) > 0

    overlap_list.append(overlap)

    # Wenn eine Überschneidung vorliegt, speichere das Projekt
    matching_project = row['Projekt'] if overlap else None
    matching_projects_list.append(matching_project)

# Füge Spalten mit den Überschneidungen, den entsprechenden Keys und Projekten zum DataFrame 2 hinzu
dfProjectsOnlinePNK['is_top_Product'] = overlap_list
dfProjectsOnlinePNK['Produktname'] = matching_keys_list

print(dfProjectsOnlinePNK)

In [ ]:
mengenanalyse_online = dfProjectsOnlinePNK
mengenanalyse_online_path ='/content/drive/My Drive/BAHAG/Data/MengenanalyseOnline.csv'
mengenanalyse_online.to_csv(mengenanalyse_online_path, index=False)

In [ ]:
# Erstelle leere Listen für die Überschneidungen, die entsprechenden Keys und Projekte
overlap_list = []
matching_keys_list = []
matching_projects_list = []

# Durchlaufe die Zeilen in df2
for index, row in dfProjectsOfflinePNK.iterrows():
    # Überprüfe auf Überschneidung und speichere die übereinstimmenden Keys
    matching_keys = [key for key in sorted_overlapPNK['Key'] if key in row['VerbundeneKnoten']]
    overlap = len(matching_keys) > 0

    overlap_list.append(overlap)
    matching_keys_list.append(matching_keys if overlap else None)

    # Wenn eine Überschneidung vorliegt, speichere das Projekt
    matching_project = row['Projekt'] if overlap else None
    matching_projects_list.append(matching_project)

# Füge Spalten mit den Überschneidungen, den entsprechenden Keys und Projekten zum DataFrame 2 hinzu
dfProjectsOfflinePNK['is_top_PNK'] = overlap_list
dfProjectsOfflinePNK['ÜbereinstimmenderKey'] = matching_keys_list

print(dfProjectsOfflinePNK)



mergedTest = pd.merge(sorted_overlapName, dfGarten, on='SAP_NAME', how='inner')


# Erstelle leere Listen für die Überschneidungen, die entsprechenden Keys und Projekte
overlap_list = []
matching_keys_list = []
matching_projects_list = []

# Durchlaufe die Zeilen in df2
for index, row in dfProjectsOfflinePNK.iterrows():
    # Überprüfe auf Überschneidung und speichere die übereinstimmenden Keys
    if row['ÜbereinstimmenderKey'] is not None:
        matching_sap_name = mergedTest.loc[mergedTest['PRODUCT_NAME_CATEGORY'].isin(row['ÜbereinstimmenderKey']), 'SAP_NAME']

        if not matching_sap_name.empty:
            matching_sap_name = matching_sap_name.iloc[0]
            overlap = True
        else:
            matching_sap_name = None
            overlap = False

        matching_keys_list.append(matching_sap_name if overlap else None)
    else:
      matching_sap_name = ""
      matching_keys_list.append(None)
      overlap = len(matching_sap_name) > 0

    overlap_list.append(overlap)

    # Wenn eine Überschneidung vorliegt, speichere das Projekt
    matching_project = row['Projekt'] if overlap else None
    matching_projects_list.append(matching_project)

# Füge Spalten mit den Überschneidungen, den entsprechenden Keys und Projekten zum DataFrame 2 hinzu
dfProjectsOfflinePNK['is_top_Product'] = overlap_list
dfProjectsOfflinePNK['Produktname'] = matching_keys_list

print(dfProjectsOfflinePNK)

In [ ]:
mengenanalyse_offline = dfProjectsOfflinePNK
mengenanalyse_offline_path ='/content/drive/My Drive/BAHAG/Data/MengenanalyseOffline.csv'
mengenanalyse_offline.to_csv(mengenanalyse_offline_path, index=False)